In [1]:
# Translated to .py by Meritxell Pacheco (December 2016)
# Adapted to PandasBiogeme by Nicola Ortelli (November 2019)

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
from biogeme.optimization import newtonTrustRegionForBiogeme
from biogeme.expressions import Beta, DefineVariable, MonteCarlo, bioDraws, log

df = pd.read_csv("swissmetro.dat", sep = '\t')
database = db.Database("swissmetro", df)

globals().update(database.variables)

# Exclude data
exclude = (( PURPOSE != 1 ) * ( PURPOSE != 3 ) + ( CHOICE == 0 )) > 0
database.remove(exclude)

# Parameters to be estimated
ASC_TRAIN    = Beta('ASC_TRAIN',    0, None, None, 1)
ASC_SM_mean  = Beta('ASC_SM_mean',  0, None, None, 0)
ASC_SM_std   = Beta('ASC_SM_std',   0, None, None, 0)
ASC_CAR_mean = Beta('ASC_CAR_mean', 0, None, None, 0)
ASC_CAR_std  = Beta('ASC_CAR_std',  0, None, None, 0)

B_TIME = Beta('B_TIME', 0, None, None, 0)
B_COST = Beta('B_COST', 0, None, None, 0)
B_HE   = Beta('B_HE',   0, None, None, 0)

# Random parameters
ASC_CAR_random = ASC_CAR_mean + ASC_CAR_std * bioDraws('ASC_CAR_random', 'NORMAL')
ASC_SM_random  = ASC_SM_mean  + ASC_SM_std  * bioDraws('ASC_SM_random',  'NORMAL')

# Definition of new variables
TRAIN_COST = DefineVariable('TRAIN_COST', TRAIN_CO * ( GA == 0 ), database)
SM_COST    = DefineVariable('SM_COST', SM_CO * ( GA == 0 ), database)

# Utilities
V_TRAIN = ASC_TRAIN      + B_TIME * TRAIN_TT + B_COST * TRAIN_COST + B_HE * TRAIN_HE
V_SM    = ASC_SM_random  + B_TIME * SM_TT    + B_COST * SM_COST    + B_HE * SM_HE
V_CAR   = ASC_CAR_random + B_TIME * CAR_TT   + B_COST * CAR_CO

V = {1: V_TRAIN, 2: V_SM, 3: V_CAR}
av = {1: TRAIN_AV, 2: SM_AV, 3: CAR_AV}

# Choice model estimation
prob = models.logit(V, av, CHOICE)
logprob = log(MonteCarlo(prob))
biogeme = bio.BIOGEME(database, logprob, numberOfDraws = 100)
biogeme.modelName = "Mixture Heteroskedastic"
results = biogeme.estimate(algorithm = newtonTrustRegionForBiogeme)

# Results
pandasResults = results.getEstimatedParameters()
print(pandasResults)
print(f"Nbr of observations: {database.getNumberOfObservations()}")
print(f"LL(0) = {results.data.initLogLike:.3f}")
print(f"LL(beta) = {results.data.logLike:.3f}")
print(f"rho bar square = {results.data.rhoBarSquare:.3g}")
print(f"Output file: {results.data.htmlFileName}")

                 Value   Std err     t-test       p-value  Rob. Std err  \
ASC_CAR_mean  0.241931  0.102749   2.354577  1.854383e-02      0.108105   
ASC_CAR_std  -0.120516  0.178242  -0.676134  4.989557e-01      0.094014   
ASC_SM_mean   0.856730  0.117650   7.282031  3.288481e-13      0.188027   
ASC_SM_std    2.994847  0.250436  11.958519  0.000000e+00      0.395805   
B_COST       -0.017199  0.001015 -16.937132  0.000000e+00      0.001544   
B_HE         -0.007739  0.001366  -5.663352  1.484443e-08      0.001342   
B_TIME       -0.016633  0.000917 -18.142383  0.000000e+00      0.001960   

              Rob. t-test  Rob. p-value  
ASC_CAR_mean     2.237927  2.522580e-02  
ASC_CAR_std     -1.281888  1.998819e-01  
ASC_SM_mean      4.556423  5.203212e-06  
ASC_SM_std       7.566479  3.841372e-14  
B_COST         -11.138384  0.000000e+00  
B_HE            -5.767899  8.026611e-09  
B_TIME          -8.486358  0.000000e+00  
Nbr of observations: 6768
LL(0) = -6964.663
LL(beta) = -5250.62

In [2]:
# likelihood ratio

-2*(-5315.386 -  -5250.620) # ~ Chi^2(0.95,2)=5.99

129.53200000000106